In [66]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [54]:
ranking = []
name = []
typ = []
episodes = []
start = []
end = []
members = []
score = []

for file_name in [f'data/mal_ranking{number}.txt' for number in range(0, 28615, 50)]:
    with open(file_name, encoding='latin-1') as file:
        text = file.read()

    soup = BeautifulSoup(text, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr', {'class': 'ranking-list'})

    for row in rows:
        ranking.append(row.find('span').text)
        name.append(row.find('div', {'class': 'di-ib clearfix'}).text)

        type_eps_time_members = row.find('div', {'class': 'information di-ib mt4'})
        multidata = type_eps_time_members.text
        multidata = multidata.strip().split('\n')
        multidata = [data.strip() for data in multidata]

        if 'TV Special' not in multidata[0]:
            typ.append(multidata[0].split(' ')[0])
            episodes.append(multidata[0].split(' ')[1][1:])

        else:
            typ.append(multidata[0].split(' ')[0] + ' ' + multidata[0].split(' ')[1])
            episodes.append(multidata[0].split(' ')[2][1:])

        start.append(multidata[1].split('-')[0])
        end.append(multidata[1].split('-')[1][1:])

        members.append(multidata[2].split(' ')[0].replace(',', ''))
        score.append(row.find('div', {'class': 'js-top-ranking-score-col di-ib al'}).text)


In [84]:
df = pd.DataFrame([ranking, name, typ, episodes, start, end, members, score]).transpose()
df.columns = ['Ranking', 'Name', 'Type', 'Episodes', 'Start', 'End', 'Members', 'Score']

df.Ranking = df.Ranking.astype(int)
df.Type = df.Type.astype('category')
df.Episodes[df.Episodes == '?'] = -1
df.Episodes = df.Episodes.astype(int)
df.Start = pd.to_datetime(df.Start)
df.End = pd.to_datetime(df.End)
df.Members = df.Members.astype(int)
df.Score[df.Score == 'N/A'] = -1.0
df.Score = df.Score.astype(float)


C:\Users\acer\AppData\Local\Temp\ipykernel_940\2539121469.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Episodes[df.Episodes == '?'] = -1
C:\Users\acer\AppData\Local\Temp\ipykernel_940\2539121469.py:6: SettingWithCopyWarning: 
A val

In [85]:
df.dtypes


Ranking              int32
Name                object
Type              category
Episodes             int32
Start       datetime64[ns]
End         datetime64[ns]
Members              int32
Score              float64
dtype: object

In [86]:
df.to_excel('data.xlsx', index=False)
